In [1]:
# Importação de bibliotecas
import numpy as np
import pandas as pd
import cantera as ct
import feedstock2 as fs
import pp2 as pp
import gasifier2 as g
import fuel as fu
import energy as en
import outputs as op

For species Fe(c), discontinuity in cp/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  0.0
	Value computed using high-temperature polynomial: 4.2881898

For species Fe(c), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  0.0
	Value computed using high-temperature polynomial: 3.425820683333331

For species Fe(c), discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  0.0
	Value computed using high-temperature polynomial: 8.486560101557188

For species Cr(cr), discontinuity in cp/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  749.1520512400002
	Value computed using high-temperature polynomial: 0.0

For species Cr(cr), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  113.24055562733336
	Value computed using high-temperature polynomial: 0.0

For species Cr(cr), discontinuity in s/R detec

In [2]:
# Bibliotecas antigas
import feedstock as fs1
import pp as pp1
import gasifier as g1

In [3]:
# Definição dos parâmetros
fuel1 = 'SMC'
fuel2 = 'RS'
T = 940 + 273.15
P = ct.one_atm
ER = 0.32
SR = 0.49/(pp.Mw['H2O']/pp.Mw['C'])
m1 = 2.4
m2 = 0.6
mt = m1+m2
moist = 0
fm1 = fs.getFuelMix(fuel1, m1/mt)
fm2 = fs.getFuelMix(fuel2, m2/mt)
bld = fs.blend(fm1, fm2)
air = fs.ERtoair(bld, ER)
steam = fs.SRtosteam(bld, SR)
f = g.getFeed(bld, 0, air, steam)
#print(fs.getFuelMass(f)/(air+steam+m1+m2))

# Gaseificação
a, b = g.isotCogasification(fuel1, fuel2, m1/mt, m2/m1, moist, T, P, air, steam, 'air', 'steam')

In [4]:
fracs = fs1.fraction([fuel1, fuel2])['mass']
fracmix = g1.blending(m2/mt, fracs[0], fracs[1])

air1 = g1.mass_of_air(fracmix, 1, ER)
print(air1)
print(air)

steam1 = g1.mass_of_steam(fracmix, 1, SR)
print(steam1)
print(steam)

2.7303258121106464
2.730325812110646
0.323174442670668
0.323174442670668


In [5]:
p1,r1 = g1.equilibrate_tp(fracmix, moist, 1, air, 0, steam, T, P)
print(p1[0] - b.species_moles[0])

2.303929616531697e-19


In [6]:
print(0, pp.i_inv[0])
print(1, pp.i_inv[1])
print(10,pp.i_inv[10])
print(15,pp.i_inv[15])
print(21,pp.i_inv[21])
print(26,pp.i_inv[26])
print(32,pp.i_inv[32])
print(40,pp.i_inv[40])
print(43,pp.i_inv[43])
print(49,pp.i_inv[49])
print(52,pp.i_inv[52])
print(56,pp.i_inv[56])
print(59,pp.i_inv[59])
print(60,pp.i_inv[60])
print(61,pp.i_inv[61])
print(62,pp.i_inv[62])
print(63,pp.i_inv[63])
print(67,pp.i_inv[67])
print(73,pp.i_inv[73])
print(76,pp.i_inv[76])
print(85,pp.i_inv[85])
print(89,pp.i_inv[89])

0 C(gr)
1 AL2O3(a)
10 CaO(s)
15 Fe2O3(s)
21 K2O(s)
26 MgO(s)
32 Na2O(c)
40 SiO2(hqz)
43 TiO2(ru)
49 Cr2O3(s)
52 C
56 H
59 H2O
60 O
61 O2
62 N
63 N2
67 S
73 SO3
76 CL
85 P2O5
89 Ar


In [7]:
# Printando os dados
print(b.species_moles[pp.i['CO2']]/sum(b.species_moles)*100)
print(b.species_moles[pp.i['H2']]/sum(b.species_moles)*100)
print(b.species_moles[pp.i['N2']]/sum(b.species_moles)*100)
print(b.species_moles[pp.i['CO']]/sum(b.species_moles)*100)
print(b.species_moles[pp.i['CH4']]/sum(b.species_moles)*100)
print(b.species_moles[pp.i['H2']]/b.species_moles[pp.i['CO']])

# fracs = a.species_moles/sum(a.species_moles)
# #print(a.species_moles)
# print(fracs[1])

OC = a.element_moles('O')/a.element_moles('C')
HC = a.element_moles('H')/a.element_moles('C')

print(OC)
print(HC)

3.889849356874009
19.829758949171172
42.935057931983394
27.94730133663259
0.0018129686654479351
0.7095411005991782
1.2934712118341203
1.4924713890441967


In [8]:
print(op.gasYield(b))
print(op.getSpecies(b))
print(b.species_moles[pp.i['AL6Si2O13(s)']]/sum(b.species_moles))

2.0161424817955003
['AL2O3(a)', 'AL2SiO5(an)', 'AL6Si2O13(s)', 'CaS(s)', 'CaO(s)', 'FeS(a)', 'K(cr)', 'MgAL2O4(s)', 'MgO(s)', 'MgSiO3(II)', 'Mg2SiO4(s)', 'NaALO2(b)', 'SiO2(hqz)', 'TiO2(ru)', 'CH4', 'CO', 'CO2', 'H2', 'H2O', 'N2', 'NH3', 'H2S', 'COS', 'P2', 'PH3', 'PO', 'PO2', 'Ar']
1.7981444699568256e-06


In [9]:
print(op.getAmounts(b, op.getSpecies(b), norm=True, db=True))
print(g1.get_fraction(p1, op.getSpecies(b), normalized='n', db='y'))

[5.61602209e-04 4.77194736e-04 1.87164565e-06 7.18076709e-04
 3.29320084e-04 4.64212942e-04 1.11534056e-03 4.23231267e-05
 2.33974169e-05 3.45943405e-04 1.34558328e-05 2.12140093e-04
 5.01763959e-03 6.95034644e-05 1.88707581e-05 2.90896788e-01
 4.04885134e-02 2.06403227e-01 4.08761287e-02 4.46900769e-01
 1.09341924e-05 2.07100637e-04 9.30645517e-06 6.70039864e-05
 2.01982570e-06 2.50901341e-06 2.61914105e-06 5.28737113e-03]
[5.35896823e-04 4.55352807e-04 1.78597759e-06 6.85209246e-04
 3.14246603e-04 4.42965210e-04 1.06428972e-03 4.03859328e-05
 2.23264816e-05 3.30109049e-04 1.28399388e-05 2.02430119e-04
 4.78797460e-03 6.63221852e-05 1.80070148e-05 2.77582000e-01
 3.86352926e-02 1.96955838e-01 3.90051661e-02 4.26445441e-01
 1.04337178e-05 1.97621326e-04 8.88048457e-06 6.39371121e-05
 1.92737521e-06 2.39417205e-06 2.49925898e-06 5.04536011e-03]


In [10]:
print(g1.carbon_conversion(p1, r1))
print(op.carbonConv(b, a))

0.9990671303963669
0.9990671303963669
